# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 2:</center>
### <center> Propiedades en Venta </center>
### <center>  Grupo 18</center>


#### **Integrantes:**

* Analía Acevedo
* Santiago Daniel Marczewski Neudeck
* Lucia Belén Napoli
* Brayan Ricaldi

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn.preprocessing as skp

from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
from preprocessing import preprocesamiento

from sklearn.model_selection import RandomizedSearchCV
from joblib import dump, load

pd.options.display.max_columns = None

from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import make_classification
from collections import defaultdict
from sklearn.model_selection import GridSearchCV

import plotly.graph_objects as go
%matplotlib inline


from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import visualkeras
from keras.wrappers.scikit_learn import KerasClassifier

np.random.seed(1)
tf.random.set_seed(1)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/home/lucia/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2022-12-08 21:49:29.541673: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 21:49:29.754332: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 21:49:29.754358: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on 

Descargo los datos para el modelo 1

In [2]:
_x_train_model = pd.read_csv('./DataSet/x_train')
_x_train_model.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model = pd.read_csv('./DataSet/y_train')
_y_train_model.drop(columns=['Unnamed: 0'],inplace=True)
_x_test_model = pd.read_csv('./DataSet/x_test')
_x_test_model.drop(columns=['Unnamed: 0'],inplace=True)
_y_test_model = pd.read_csv('./DataSet/y_test')
_y_test_model.drop(columns=['Unnamed: 0'],inplace=True)
_y_train_model.rename(columns={'0':'pxm2'}, inplace=True)
_y_test_model.rename(columns={'0':'pxm2'}, inplace=True)

# Redes neuronales

Construir dos modelos de redes neuronales, uno para regresión y otro para clasificación,
considerando los datasets creados en el TP1. Mejorar estos modelos de redes
neuronales a través de la búsqueda de arquitectura e hiperparámetros adecuados. Se
pide:

● Regresión: predecir el precio de la propiedad y utilizar como métrica de
evaluación el error cuadrático medio.

● Clasificación: predecir el atributo tipo_precio creado en el TP 1 y utilizar como
métricas precisión, recall y F1-Score.


# Modelo de Redes Neuronales para regresión

Los hiperparametros a buscar son los siguientes:

- **Función de activación:** relu, tanh
- **Cantidad de capas:** Es la cantidad de capas que tendrá la red neuronal. Tendremos una capa input, x ocultas y una capa de salida.
- **Cantidad de neuronas por capa:** Tendremos una neurona en la capa de salida porque queremos predecir un valor numerico. En la capa de entrada tendremos 8 neuronas, la cantidad de features que tiene el dataset.
- **Optimizador:** Un optimizador es una implementación concreta del algoritmo de descenso por gradiente dentro de backpropagation.
    - SGD (Stochastic Gradient Descent): Backpropagation simple, sin ningún tipo de optimización. 
    - Adam (Adaptive moment estimation): Hace un seguimiento de una media de decaimiento exponencial de gradientes pasados y de gradientes cuadrados pasados.
- **Función de perdida:** Es la función utilizada por el optimizador para minimizar. En regresion utilizaremos Mean Absolute Error (MAE)
- **Learning rate o tasa de aprendizaje:** Indica qué tan rápido se va descediendo en la función de costo, qué tan rápido va acercandose al minimo local o global.
- **Metricas:** Se van calculando en cada iteracion o batch. Utilizaremos Mean Squared Error (MSE) y Mean Absolute Error (MAE).
- **Cantidad de epochs:** Es la cantidad de iteraciones que tendrá el entrenamiento.

Ejecutamos una copia de los modelos

In [3]:
x_train = _x_train_model.copy()
y_train = _y_train_model.copy()
x_test = _x_test_model.copy()
y_test = _y_test_model.copy()

In [4]:
x_train.head()

,latitud,longitud,ambientes,dormitorios,superficie_cubierta,comuna,Departamento,PH
0,-34.583010,-58.481533,4.0,3.0,70.0,15,1.0,0.0
1,-34.660079,-58.485894,6.0,4.0,275.0,9,0.0,0.0
2,-34.581856,-58.425088,3.0,2.0,97.0,14,1.0,0.0
3,-34.607547,-58.442194,2.0,1.0,42.0,6,1.0,0.0
4,-34.607416,-58.509680,2.0,1.0,38.0,11,1.0,0.0


In [5]:
y_train.head()

,pxm2
0,2911.392405
1,671.428571
2,2546.391753
3,1733.333333
4,2804.878049


In [6]:
x_train.shape

(58133, 8)

Normalizo datos

In [7]:
x_train_red = x_train.iloc[:, 0:6]
x_train_norm = (x_train_red - x_train_red.mean())/x_train_red.std()
x_train_norm.head()

,latitud,longitud,ambientes,dormitorios,superficie_cubierta,comuna
0,0.501718,-1.122143,0.895188,1.112654,-0.086764,1.271560
1,-2.412422,-1.237399,2.384149,2.110384,3.243882,0.044010
2,0.545327,0.369634,0.150707,0.114923,0.351906,1.066968
3,-0.426107,-0.082471,-0.593774,-0.882808,-0.541681,-0.569765
4,-0.421158,-1.866017,-0.593774,-0.882808,-0.606670,0.453193


In [8]:
x_test_red = x_test.iloc[:, 0:6]
x_test_norm = (x_test_red - x_test_red.mean())/x_test_red.std()
x_test_norm.head()

,latitud,longitud,ambientes,dormitorios,superficie_cubierta,Departamento
0,0.736100,0.852283,0.871830,1.091760,1.474945,0.415401
1,1.964042,-0.548882,0.871830,1.091760,0.342862,0.415401
2,0.174311,1.184193,0.871830,1.091760,0.703816,0.415401
3,-1.233172,0.169444,-1.309095,-0.854592,-0.690780,0.415401
4,0.170750,1.617522,0.144855,0.118584,1.327282,0.415401


+ Transformo los set de datos

In [9]:
sscaler = StandardScaler()
sscaler.fit(pd.DataFrame(x_train_norm))

StandardScaler()

In [10]:
x_train_transform = sscaler.transform(pd.DataFrame(x_train_norm))
x_test_transform = sscaler.transform(pd.DataFrame(x_test_norm))

In [11]:
x_train_transform[0:5]

array([[ 0.50172268, -1.12215239,  0.89519544,  1.11266314, -0.0867647 ,
         1.27157065],
       [-2.41244287, -1.23741004,  2.38416976,  2.11040252,  3.24390971,
         0.04401013],
       [ 0.54533212,  0.36963696,  0.15070829,  0.11492376,  0.35190949,
         1.06697723],
       [-0.42611094, -0.08247136, -0.59377887, -0.88281562, -0.54168608,
        -0.56977013],
       [-0.42116132, -1.86603329, -0.59377887, -0.88281562, -0.60667485,
         0.45319697]])

# Con dos capas

## Modelo 1

In [14]:
d_in=6
d_out=1

modelo_1 = keras.Sequential([
    keras.layers.Dense(6,input_shape=(d_in,), activation="relu"),
    
    keras.layers.Dense(d_out)])

In [15]:
modelo_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 6)                 42        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [16]:
modelo_1.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01), 
      loss='mae', 
      metrics=['mse','mae'], 
)

In [17]:
# Entrenamiento del modelo
modelo_1.fit(x_train_transform,y_train,epochs=50,batch_size=10,verbose=2)

Epoch 1/50
5814/5814 - 5s - loss: 676.1497 - mse: 1143967.3750 - mae: 676.1497 - 5s/epoch - 778us/step
Epoch 2/50
5814/5814 - 4s - loss: 537.6767 - mse: 707642.9375 - mae: 537.6767 - 4s/epoch - 717us/step
Epoch 3/50
5814/5814 - 4s - loss: 532.7826 - mse: 697072.5000 - mae: 532.7826 - 4s/epoch - 701us/step
Epoch 4/50
5814/5814 - 4s - loss: 528.2028 - mse: 690834.2500 - mae: 528.2027 - 4s/epoch - 704us/step
Epoch 5/50
5814/5814 - 4s - loss: 526.5625 - mse: 687893.8125 - mae: 526.5625 - 4s/epoch - 693us/step
Epoch 6/50
5814/5814 - 4s - loss: 526.5090 - mse: 687842.6250 - mae: 526.5090 - 4s/epoch - 681us/step
Epoch 7/50
5814/5814 - 4s - loss: 525.8823 - mse: 688373.1875 - mae: 525.8823 - 4s/epoch - 691us/step
Epoch 8/50
5814/5814 - 4s - loss: 524.9023 - mse: 683293.8125 - mae: 524.9023 - 4s/epoch - 680us/step
Epoch 9/50
5814/5814 - 4s - loss: 523.1291 - mse: 687042.8125 - mae: 523.1291 - 4s/epoch - 702us/step
Epoch 10/50
5814/5814 - 4s - loss: 520.6960 - mse: 680641.9375 - mae: 520.6960 - 

In [18]:
y_pred = modelo_1.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 745us/step
Error absoluto medio 546.3714164951018
Error cuadrático medio 612587.9254594041


## Modelo 2

Cambio learning rate

In [19]:
d_in=6
d_out=1

modelo_2 = keras.Sequential([
    keras.layers.Dense(6,input_shape=(d_in,), activation="relu"),
    
    keras.layers.Dense(d_out)])

In [20]:
modelo_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6)                 42        
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [21]:
modelo_2.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.001), 
      loss='mae', 
      metrics=['mse','mae'], 
)

In [22]:
# Entrenamiento del modelo
modelo_2.fit(x_train_transform,y_train,epochs=50,batch_size=10,verbose=2)

Epoch 1/50
5814/5814 - 4s - loss: 1922.6705 - mse: 5013111.0000 - mae: 1922.6705 - 4s/epoch - 751us/step
Epoch 2/50
5814/5814 - 4s - loss: 565.9844 - mse: 800535.7500 - mae: 565.9844 - 4s/epoch - 701us/step
Epoch 3/50
5814/5814 - 4s - loss: 565.3915 - mse: 794925.0625 - mae: 565.3915 - 4s/epoch - 710us/step
Epoch 4/50
5814/5814 - 4s - loss: 565.2048 - mse: 792485.5625 - mae: 565.2048 - 4s/epoch - 703us/step
Epoch 5/50
5814/5814 - 4s - loss: 565.2154 - mse: 793979.1875 - mae: 565.2154 - 4s/epoch - 705us/step
Epoch 6/50
5814/5814 - 4s - loss: 565.1254 - mse: 792572.0625 - mae: 565.1254 - 4s/epoch - 716us/step
Epoch 7/50
5814/5814 - 4s - loss: 564.8787 - mse: 792370.6250 - mae: 564.8787 - 4s/epoch - 710us/step
Epoch 8/50
5814/5814 - 4s - loss: 564.2918 - mse: 790889.9375 - mae: 564.2918 - 4s/epoch - 712us/step
Epoch 9/50
5814/5814 - 4s - loss: 563.4897 - mse: 790016.6250 - mae: 563.4897 - 4s/epoch - 723us/step
Epoch 10/50
5814/5814 - 4s - loss: 561.9044 - mse: 785108.5000 - mae: 561.9044 

In [18]:
y_pred = modelo_1.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 745us/step
Error absoluto medio 546.3714164951018
Error cuadrático medio 612587.9254594041


Obtengo mismo resultado

## Modelo 3

Optimizador adam

In [11]:
d_in=6
d_out=1

modelo_3 = keras.Sequential([
    keras.layers.Dense(6,input_shape=(d_in,), activation="relu"),
    
    keras.layers.Dense(d_out)])

2022-12-08 20:12:02.995598: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-08 20:12:02.995630: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-08 20:12:02.995659: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lucia-Ubuntu-laptop): /proc/driver/nvidia/version does not exist
2022-12-08 20:12:02.995980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
modelo_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 42        
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [13]:
modelo_3.compile(
      optimizer=keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999), 
      loss='mae', 
      metrics=['mse','mae'], 
)

In [14]:
# Entrenamiento del modelo
modelo_3.fit(x_train_transform,y_train,epochs=50,batch_size=10,verbose=2)

Epoch 1/50
5814/5814 - 5s - loss: 841.8068 - mse: 1556017.1250 - mae: 841.8068 - 5s/epoch - 806us/step
Epoch 2/50
5814/5814 - 4s - loss: 561.5845 - mse: 791761.5000 - mae: 561.5845 - 4s/epoch - 736us/step
Epoch 3/50
5814/5814 - 4s - loss: 556.6748 - mse: 773541.3125 - mae: 556.6747 - 4s/epoch - 749us/step
Epoch 4/50
5814/5814 - 4s - loss: 554.5261 - mse: 765022.0625 - mae: 554.5261 - 4s/epoch - 741us/step
Epoch 5/50
5814/5814 - 4s - loss: 551.9435 - mse: 757135.3125 - mae: 551.9435 - 4s/epoch - 741us/step
Epoch 6/50
5814/5814 - 4s - loss: 549.6460 - mse: 751141.9375 - mae: 549.6460 - 4s/epoch - 751us/step
Epoch 7/50
5814/5814 - 4s - loss: 546.5311 - mse: 746206.0000 - mae: 546.5311 - 4s/epoch - 745us/step
Epoch 8/50
5814/5814 - 4s - loss: 539.2139 - mse: 736367.6875 - mae: 539.2139 - 4s/epoch - 731us/step
Epoch 9/50
5814/5814 - 4s - loss: 534.3906 - mse: 729132.5000 - mae: 534.3906 - 4s/epoch - 733us/step
Epoch 10/50
5814/5814 - 4s - loss: 531.4589 - mse: 721672.6875 - mae: 531.4589 - 

In [16]:
y_pred = modelo_3.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 729us/step
Error absoluto medio 582.6529809429456
Error cuadrático medio 652750.5472689988


# Con tres capas

## Modelo 4

In [55]:
d_in=6
d_out=1

modelo_4 = keras.Sequential([
    
    keras.layers.Dense(6,input_shape=(d_in,), activation="relu"),
    #keras.layers.Dense(6,input_shape=(d_in,), activation="relu", kernel_initializer='normal'),
    
    keras.layers.Dense(6, activation="relu"),
    
    keras.layers.Dense(d_out)])

In [47]:
modelo_4.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 6)                 42        
                                                                 
 dense_26 (Dense)            (None, 1)                 7         
                                                                 
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [49]:
modelo_4.compile(
      #optimizer=keras.optimizers.SGD(learning_rate=0.01), 
      optimizer=keras.optimizers.RMSprop(learning_rate=0.0001, rho=0.9), 
      loss='mae', 
      metrics=['mse','mae'], 
)

In [52]:
# Entrenamiento del modelo
#modelo_4.fit(x_train_transform,y_train,epochs=100,batch_size=10,verbose=2)
modelo_4.fit(x_train_transform,y_train,epochs=100,batch_size=32,verbose=2)

Epoch 1/100
1817/1817 - 1s - loss: 561.8891 - mse: 786407.4375 - mae: 561.8891 - 1s/epoch - 806us/step
Epoch 2/100
1817/1817 - 1s - loss: 561.7044 - mse: 784973.3125 - mae: 561.7044 - 1s/epoch - 747us/step
Epoch 3/100
1817/1817 - 1s - loss: 561.5707 - mse: 784731.3125 - mae: 561.5707 - 1s/epoch - 743us/step
Epoch 4/100
1817/1817 - 1s - loss: 561.4315 - mse: 783341.1250 - mae: 561.4315 - 1s/epoch - 790us/step
Epoch 5/100
1817/1817 - 1s - loss: 561.3050 - mse: 783685.3125 - mae: 561.3050 - 1s/epoch - 760us/step
Epoch 6/100
1817/1817 - 1s - loss: 561.1725 - mse: 783254.6875 - mae: 561.1725 - 1s/epoch - 769us/step
Epoch 7/100
1817/1817 - 1s - loss: 561.0247 - mse: 782685.3125 - mae: 561.0247 - 1s/epoch - 757us/step
Epoch 8/100
1817/1817 - 1s - loss: 560.8830 - mse: 782792.5625 - mae: 560.8830 - 1s/epoch - 754us/step
Epoch 9/100
1817/1817 - 1s - loss: 560.7208 - mse: 782341.4375 - mae: 560.7208 - 1s/epoch - 756us/step
Epoch 10/100
1817/1817 - 1s - loss: 560.6011 - mse: 781142.1250 - mae: 56

In [53]:
y_pred = modelo_4.predict(x_test_transform)

mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)

print(f"Error absoluto medio {mae}")
print(f"Error cuadrático medio {mse}")

455/455 [==============================] - 0s 765us/step
Error absoluto medio 547.8497718569255
Error cuadrático medio 661041.217672852
